In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
import pickle

In [2]:
#Read csv file of final train data into dataframe
df_train=pd.read_csv('train_final.csv',index_col=False)
labels=df_train[['784']]

In [3]:
df_train.drop(df_train.columns[[784]],axis=1,inplace=True)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#To avoid tensorflow keras version error set available gpus in your local system
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback

print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus


tf.__version__ is 2.1.0
tf.keras.__version__ is: 2.2.4-tf


In [20]:
np.random.seed(1212)
import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers
from keras.layers import Input, Dense
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
#K.set_image_dim_ordering('th')
K.set_image_data_format('channels_first')

In [21]:

labels=np.array(labels)


In [22]:
from keras.utils.np_utils import to_categorical
cat=to_categorical(labels,num_classes=13)

In [23]:
print(cat[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [24]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
l=[]
for i in range(47504):
    l.append(np.array(df_train[i:i+1]).reshape(1,28,28))

In [26]:
np.random.seed(7)

## Building CNN Model to recognize handwritten digits & symbols

In [27]:
#Model Creation
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(1 , 28, 28), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(13, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [28]:
from keras.models import model_from_json

In [29]:
model.fit(np.array(l), cat, epochs=10, batch_size=200,shuffle=True,verbose=1)

Epoch 1/10
47504/47504 [==============================] - 50s 1ms/step - loss: 1.5721 - accuracy: 0.6293
Epoch 2/10
47504/47504 [==============================] - 45s 937us/step - loss: 0.3348 - accuracy: 0.8991
Epoch 3/10
47504/47504 [==============================] - 49s 1ms/step - loss: 0.1978 - accuracy: 0.9401
Epoch 4/10
47504/47504 [==============================] - 108s 2ms/step - loss: 0.1465 - accuracy: 0.9556
Epoch 5/10
47504/47504 [==============================] - 156s 3ms/step - loss: 0.1097 - accuracy: 0.9669
Epoch 6/10
47504/47504 [==============================] - 152s 3ms/step - loss: 0.0913 - accuracy: 0.9728
Epoch 7/10
47504/47504 [==============================] - 154s 3ms/step - loss: 0.0776 - accuracy: 0.9762
Epoch 8/10
47504/47504 [==============================] - 149s 3ms/step - loss: 0.0720 - accuracy: 0.9779
Epoch 9/10
47504/47504 [==============================] - 151s 3ms/step - loss: 0.0589 - accuracy: 0.9820
Epoch 10/10
47504/47504 [======================

In [30]:
# Save model to jason file
model_json = model.to_json()
with open("model_final.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_final.h5")